In [1]:
!git clone https://github.com/HeliosZhao/Shot-Boundary-Detection.git

Cloning into 'Shot-Boundary-Detection'...
remote: Enumerating objects: 35, done.
remote: Total 35 (delta 0), reused 0 (delta 0), pack-reused 35
Unpacking objects: 100% (35/35), done.


In [4]:
%cd /content/Shot-Boundary-Detection

/content/Shot-Boundary-Detection


In [ ]:
from findframe import *
import cv2
import numpy as np
import sys


if __name__ == "__main__":
    # dataload
    videopath = sys.argv[1]
    cap = cv2.VideoCapture(str(videopath))
    curr_frame = None
    prev_frame = None
    frame_diffs = []
    frames = []
    success, frame = cap.read()
    i = 0
    FRAME = Frame(0, 0)
    while (success):
        luv = cv2.cvtColor(frame, cv2.COLOR_BGR2LUV)
        curr_frame = luv
        """
        
        calculate the difference between frames 
        
        """

        if curr_frame is not None and prev_frame is not None:
            diff = cv2.absdiff(curr_frame, prev_frame)
            diff_sum = np.sum(diff)
            diff_sum_mean = diff_sum / (diff.shape[0] * diff.shape[1])
            frame_diffs.append(diff_sum_mean)
            frame = Frame(i, diff_sum_mean)
            frames.append(frame)
        elif curr_frame is not None and prev_frame is None:
            diff_sum_mean = 0
            frame_diffs.append(diff_sum_mean)
            frame = Frame(i, diff_sum_mean)
            frames.append(frame)

        prev_frame = curr_frame
        i = i + 1
        success, frame = cap.read()
    cap.release()


    #detect the possible frame
    frame_return, start_id_spot_old, end_id_spot_old = FRAME.find_possible_frame(frames)

    #optimize the possible frame
    new_frame, start_id_spot, end_id_spot = FRAME.optimize_frame(frame_return, frames)

    #store the result
    start = np.array(start_id_spot)[np.newaxis, :]
    end = np.array(end_id_spot)[np.newaxis, :]
    spot = np.concatenate((start.T, end.T), axis=1)
    np.savetxt('./result.txt', spot, fmt='%d', delimiter='\t')


In [3]:
!ls

 findframe.py   main.py   README.md   result.txt  'shot result.png'


In [6]:
#!python main.py /content/Tom_and_jerry.mp4
#!python main.py /content/anni005.mpg
!python main.py /content/v1.mp4

In [14]:
#GROUND TRUTH TO COMPUTABLE ARRAY
import csv
import numpy as np
a = []
with open ('/content/v1_23553.txt', 'r') as f:
    first_column = [row[0] for row in csv.reader(f,delimiter='\t')]
    a.append(first_column)
print (first_column[0])
print(a)
postdata = np.array(a)
print(postdata.shape)
flat_array = postdata.flatten()
print(flat_array.shape)
print(flat_array)

b = []
with open ('/content/v1_23553.txt', 'r') as f:
    first_column = [row[1] for row in csv.reader(f,delimiter='\t')]
    b.append(first_column)
print (first_column[0])
print(b)
postdata1 = np.array(b)
print(postdata1.shape)
flat_array1 = postdata1.flatten()
print(flat_array1.shape)
print(flat_array1)

b = np.stack((flat_array, flat_array1), axis = 1)
print(b)
print(b.shape)
print(int(b[0][0]))
print((b[0][1]))


0
[['0', '523', '543', '608', '642', '717', '764', '823', '872', '918', '964', '1017', '1067', '1123', '1168', '1225', '1274', '1328', '1590', '1640', '1686', '1734', '1889', '1936', '1988', '2041', '2088', '2138', '2189', '2236', '2285', '2333', '2385', '2430', '2488', '2547', '2594', '2651', '2698', '2733', '2810', '3078', '3091', '3168', '3181', '3252', '3265', '3337', '3350', '3433', '3446', '3531', '3544', '3701', '3743', '3794', '3851', '4040', '4053', '4156', '4169', '4284', '4339', '4388', '4437', '4487', '4535', '4691', '4768', '4808', '4870', '5077', '5172', '5576', '5639', '5757', '5869', '6002', '6499', '6546', '6612', '6682', '6738', '6895', '6963', '7034', '7083', '7204', '7267', '7917', '7990', '8050', '8153', '8208', '8304', '9213', '9268', '9327', '9439', '9512', '9611', '9664', '9730', '9929', '10120', '10238', '10296', '11073', '11162', '11229', '11586', '11619', '12024']]
(1, 113)
(113,)
['0' '523' '543' '608' '642' '717' '764' '823' '872' '918' '964' '1017'
 '1067'

In [41]:
def get_transition_truths(gt):

    transition_truths = []
    for idx, truth in enumerate(gt):
        if idx < len(gt) - 1:
            transition_truths.append([truth[1], gt[idx+1][0]])
    return transition_truths
print(len(get_transition_truths(b)))
#print(get_transition_truths(b))

tt = np.array(get_transition_truths(b))
print(tt)
print(tt.shape)
print(tt[1][1])
ttr = []
for i in range(0,len(tt)):
  ttr.append(int(tt[i][1])-int(tt[i][0]))
print(ttr)

#total transitions
tot_trans = np.sum(ttr)
print(tot_trans)

#transitions matching the groundtruth



def get_precision(prediction, gt):

    success = 1
    for truth in gt:
        if prediction > int(truth[0]) and prediction <= int(truth[1]):
            success = 0
            return success
        else: 
            success = 1
    return success
#print(get_precision(pred, b))


112
[['509' '523']
 ['531' '543']
 ['597' '608']
 ['619' '642']
 ['716' '717']
 ['763' '764']
 ['822' '823']
 ['871' '872']
 ['917' '918']
 ['963' '964']
 ['1016' '1017']
 ['1066' '1067']
 ['1122' '1123']
 ['1167' '1168']
 ['1224' '1225']
 ['1273' '1274']
 ['1327' '1328']
 ['1589' '1590']
 ['1639' '1640']
 ['1685' '1686']
 ['1733' '1734']
 ['1888' '1889']
 ['1935' '1936']
 ['1987' '1988']
 ['2040' '2041']
 ['2087' '2088']
 ['2137' '2138']
 ['2188' '2189']
 ['2235' '2236']
 ['2284' '2285']
 ['2332' '2333']
 ['2384' '2385']
 ['2429' '2430']
 ['2487' '2488']
 ['2546' '2547']
 ['2593' '2594']
 ['2650' '2651']
 ['2697' '2698']
 ['2732' '2733']
 ['2796' '2810']
 ['3063' '3078']
 ['3076' '3091']
 ['3153' '3168']
 ['3167' '3181']
 ['3237' '3252']
 ['3250' '3265']
 ['3322' '3337']
 ['3335' '3350']
 ['3418' '3433']
 ['3431' '3446']
 ['3516' '3531']
 ['3529' '3544']
 ['3700' '3701']
 ['3742' '3743']
 ['3793' '3794']
 ['3850' '3851']
 ['4025' '4040']
 ['4038' '4053']
 ['4141' '4156']
 ['4154' '416

In [11]:
#PREDICTION CREATION FOR TRANSNET
import csv
import numpy as np
a = []
af = []
with open ('/content/Shot-Boundary-Detection/result.txt', 'r') as f:
    first_column = [row[0] for row in csv.reader(f,delimiter='\t')]
    a.append(first_column)
print (first_column[0])
print(a)

print(af)
postdata = np.array(a)
print(postdata.shape)
flat_array = postdata.flatten()
print(flat_array.shape)
print(flat_array)

for i in range(1,len(flat_array)):
  af.append(flat_array[i])
print(af)

afinal = np.array(af)
print(afinal)
np.savetxt('/content/resultv1_trans.txt', afinal, fmt='%s', delimiter='\t')


0
[['0', '717', '764', '823', '872', '918', '964', '1017', '1067', '1123', '1168', '1225', '1274', '1328', '1590', '1640', '1686', '1889', '1936', '1988', '2041', '2088', '2138', '2189', '2236', '2285', '2333', '2385', '2430', '2488', '2547', '2594', '2651', '2698', '2733', '3701', '3743', '3794', '3851', '4284', '4339', '4388', '4437', '4487', '4535', '4691', '4768', '4808', '4870', '5077', '5576', '5639', '5757', '5869', '6002', '6499', '6546', '6556', '6612', '6682', '6738', '6895', '6963', '7034', '7083', '7204', '7267', '7917', '7990', '8050', '8153', '8208', '8304', '9213', '9268', '9327', '9439', '9512', '9611', '9664', '9730', '9929', '10120', '10238', '10269', '10296', '11073', '11162', '11229']]
[]
(1, 89)
(89,)
['0' '717' '764' '823' '872' '918' '964' '1017' '1067' '1123' '1168'
 '1225' '1274' '1328' '1590' '1640' '1686' '1889' '1936' '1988' '2041'
 '2088' '2138' '2189' '2236' '2285' '2333' '2385' '2430' '2488' '2547'
 '2594' '2651' '2698' '2733' '3701' '3743' '3794' '3851' 

In [15]:
#PREDICTIONS TO COMPUTABLE ARRAY
import csv
import numpy as np

import csv
import numpy as np
a = []
with open ('/content/predv1_23553_gygli.txt', 'r') as f:
    first_column = [row[0] for row in csv.reader(f,delimiter='\t')]
    a.append(first_column)
#print (first_column[0])
#print(a)
postdata = np.array(a)
print(postdata.shape)
flat_array = postdata.flatten()
pred = flat_array
print(pred.shape)
print(len(pred))



(1, 314)
(314,)
314


In [20]:
#CODE FOR PRECISION RECALL AND F1 SCORE CALCULATION
def get_transition_truths(gt):

    transition_truths = []
    for idx, truth in enumerate(gt):
        if idx < len(gt) - 1:
            transition_truths.append([truth[1], gt[idx+1][0]])
    return transition_truths

print(len(get_transition_truths(b)))
#print(get_transition_truths(b))

tt = np.array(get_transition_truths(b))
print(tt)
print(tt.shape)
print(tt[1][1])
ttr = []
for i in range(0,len(tt)):
  ttr.append(int(tt[i][1])-int(tt[i][0]))
print(ttr)

#total transitions
tot_trans = np.sum(ttr)
print(tot_trans)

#transitions matching the groundtruth


def get_precision(prediction, gt):

    success = 1
    for truth in gt:
        if prediction > int(truth[0]) and prediction < int(truth[1]):
            success = 0
            return success
        else: 
            success = 1
    return success

def get_recall(prediction, gt):

    success = 0
    for truth in gt:
        if prediction > int(truth[0]) and prediction <= int(truth[1]):
            return 1
        else:
            success = 0
    return success

def getF1(p, r):

    a = 2 * (p * r)
    b = p + r
    return a / b 


print(get_transition_truths(b))
print(pred)
#PRECISION
pre1 = []
for i in range(0,len(pred)):
  pre1.append(get_precision(int(pred[i]),b))
print(pre1)
p_value = np.mean(np.array(pre1))
print("precision")
print(p_value)

#RECALL

re1 = []
for i in range(0,len(pred)):#pred
  re1.append(get_recall(int(pred[i]),get_transition_truths(b)))
print(re1)
#r_value = np.mean(np.array(re1))
r_value = np.count_nonzero(np.array(re1))
print("***")
print(r_value)
recall1 = r_value/len(get_transition_truths(b))
#recall1 = r_value/tot_trans
print("recall")
#print(r_value)
print(recall1)

#F1 SCORE
print("F1 SCORE")
#print(getF1(p_value, r_value))
print(getF1(p_value, recall1))



#print(get_precision(342,b))



112
[['509' '523']
 ['531' '543']
 ['597' '608']
 ['619' '642']
 ['716' '717']
 ['763' '764']
 ['822' '823']
 ['871' '872']
 ['917' '918']
 ['963' '964']
 ['1016' '1017']
 ['1066' '1067']
 ['1122' '1123']
 ['1167' '1168']
 ['1224' '1225']
 ['1273' '1274']
 ['1327' '1328']
 ['1589' '1590']
 ['1639' '1640']
 ['1685' '1686']
 ['1733' '1734']
 ['1888' '1889']
 ['1935' '1936']
 ['1987' '1988']
 ['2040' '2041']
 ['2087' '2088']
 ['2137' '2138']
 ['2188' '2189']
 ['2235' '2236']
 ['2284' '2285']
 ['2332' '2333']
 ['2384' '2385']
 ['2429' '2430']
 ['2487' '2488']
 ['2546' '2547']
 ['2593' '2594']
 ['2650' '2651']
 ['2697' '2698']
 ['2732' '2733']
 ['2796' '2810']
 ['3063' '3078']
 ['3076' '3091']
 ['3153' '3168']
 ['3167' '3181']
 ['3237' '3252']
 ['3250' '3265']
 ['3322' '3337']
 ['3335' '3350']
 ['3418' '3433']
 ['3431' '3446']
 ['3516' '3531']
 ['3529' '3544']
 ['3700' '3701']
 ['3742' '3743']
 ['3793' '3794']
 ['3850' '3851']
 ['4025' '4040']
 ['4038' '4053']
 ['4141' '4156']
 ['4154' '416

In [ ]:
#ORIGINAL CSV FILE CREATION CODE FOR TRAINING A NETWORK
from findframe import *
import cv2
import numpy as np
import sys


if __name__ == "__main__":
    # dataload
    videopath = sys.argv[1]
    cap = cv2.VideoCapture(str(videopath))
    curr_frame = None
    prev_frame = None
    frame_diffs = []
    frames = []
    fno = []
    fno.append('Image_ID')
    prev = []
    post = []
    prev.append('prev')
    post.append('post')

    fno_int = []
    fno_int.append(1)
    classi = []
    classi.append('Class')
    a = []
    success, frame = cap.read()
    i = 0
    FRAME = Frame(0, 0)
    while (success):
        luv = cv2.cvtColor(frame, cv2.COLOR_BGR2LUV)
        curr_frame = luv
        """
        
        calculate the difference between frames 
        
        """

        if curr_frame is not None and prev_frame is not None:
            diff = cv2.absdiff(curr_frame, prev_frame)
            diff_sum = np.sum(diff)
            diff_sum_mean = diff_sum / (diff.shape[0] * diff.shape[1])
            #print(diff_sum_mean)
            frame_diffs.append(diff_sum_mean)
            frame = Frame(i, diff_sum_mean)
            frames.append(frame)
            
        elif curr_frame is not None and prev_frame is None:
            diff_sum_mean = 0
            #print(diff_sum_mean)
            frame_diffs.append(diff_sum_mean)
            frame = Frame(i, diff_sum_mean)
            frames.append(frame)
            
        prev_frame = curr_frame
        print(i)
        filename ="frame%d.jpg" % i
        fno.append(filename)
        fno_int.append(i)
        #classi.append(0)
        i = i + 1
        success, frame = cap.read()
    cap.release()
    print(fno)
    print(frame_diffs)
    print(frame_diffs[71])
    print(frame_diffs[72])
    print(frame_diffs[73])
    print(frame_diffs[158])
    print(frame_diffs[159])
    print(frame_diffs[160])

    #detect the possible frame
    frame_return, start_id_spot_old, end_id_spot_old = FRAME.find_possible_frame(frames)

    #optimize the possible frame
    new_frame, start_id_spot, end_id_spot = FRAME.optimize_frame(frame_return, frames)

    #store the result
    start = np.array(start_id_spot)[np.newaxis, :]
    end = np.array(end_id_spot)[np.newaxis, :]
    #spot = np.concatenate((start.T, end.T), axis=1)
    spot = np.concatenate((start.T), axis=0)
    spot1 = np.concatenate((end.T), axis=0)
    print(spot)
    print(spot1)
    print(len(spot))
    print(len(fno))
    print(fno)
    print(classi)



    
    
    for i in range(len(spot)):
      while spot[i] < spot1[i]:
        prev.append("frame%d.jpg" % spot[i])
        spot[i] = spot[i]+1
        post.append("frame%d.jpg" % spot[i])
        classi.append(0)
      if (spot[i] < spot1[len(spot)-1]):
        prev.append("frame%d.jpg" % spot[i])
        post.append("frame%d.jpg" % (spot[i]+1))
        classi.append(1)
      else:
        break
    print(prev)
    print(post)
    print(classi)

    prevdata = np.array(prev)
    print(prevdata.shape)
    postdata = np.array(post)
    print(postdata.shape)
    clas = np.array(classi)
    print(clas.shape)
    feat = np.stack((prevdata, postdata, clas), axis = 1)
    print(feat)
    np.savetxt('/content/Shot-Boundary-Detection/result.csv', feat, fmt='%s', delimiter=',')
